# CART-Microbiome: exploratory data-analyses

### Importing libraries

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
import seaborn as sns

from scipy.stats import mannwhitneyu, wilcoxon
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import make_interp_spline

from skbio.diversity.alpha import shannon
from skbio.stats.distance import permanova
from skbio import DistanceMatrix
from skbio.stats.ordination import pcoa

from statsmodels.stats.multitest import fdrcorrection
from matplotlib.legend_handler import HandlerBase


### Defining directories

In [ ]:
path_input = "../../CART_Microbiome_input/"
path_output = "../../CART_Microbiome_output/"


### Reading and processing the input tables

In [ ]:
df_CART_table_stool =      pd.read_excel(path_input + 'CART_table_stool.xlsx')
df_CART_table_taxa =       pd.read_excel(path_input + 'CART_table_taxa.xlsx')
df_CART_table_pathways =   pd.read_excel(path_input + 'CART_table_pathways.xlsx')
df_CART_table_genes =      pd.read_excel(path_input + 'CART_table_genes.xlsx')
df_CART_table_patients =   pd.read_excel(path_input + 'CART_table_patients.xlsx')
df_CART_table_response =   pd.read_excel(path_input + 'CART_table_response.xlsx')
df_CART_table_blood =      pd.read_excel(path_input + 'CART_table_blood.xlsx')
CR_coefficients_df =       pd.read_excel(path_input + 'CR180_noHighRiskABX_elasticnet_train_coefficients.xlsx')
df_kdt_numbers =           pd.read_excel(path_input + 'CART_postkdt_reads.xlsx')


df_CART_table_stool.index = df_CART_table_stool.stool_sample_id
df_CART_table_taxa.index = df_CART_table_taxa.stool_sample_id
df_CART_table_pathways.index = df_CART_table_pathways.stool_sample_id
df_CART_table_genes.index = df_CART_table_genes.stool_sample_id

df_CART_table_taxa = df_CART_table_taxa.drop(["stool_sample_id"], axis = 1)
df_CART_table_pathways = df_CART_table_pathways.drop(["stool_sample_id","UNMAPPED", "UNINTEGRATED"], axis = 1)
df_CART_table_genes = df_CART_table_genes.drop(["stool_sample_id","UNMAPPED", "UNGROUPED"], axis = 1)


# Identifying patients who have a mixture of both HR-Abx and none-|LR-Abx basal samples:

df_CART_table_stool_post_basal_removed = df_CART_table_stool.loc[(df_CART_table_stool["sampleDay_sample"] < 1) &  (df_CART_table_stool["sampleDay_sample"] > -22),:]
patient_mixed_basal = (df_CART_table_stool_post_basal_removed.groupby("subjectID")["Basal_no_HighRisk_sample"].apply(lambda x:x.value_counts()).to_frame().groupby(level="subjectID").size()==2)
patient_mixed_basal_true = list(patient_mixed_basal[patient_mixed_basal ==True].index)


# Identifying samples that have < 5e5 post-KneadData reads:

df_kdt_numbers = df_kdt_numbers[~df_kdt_numbers.FileName.str.contains("paired_2")] 
df_kdt_numbers['FileName'] = df_kdt_numbers['FileName'].str.split("_combined_R1_001_kneaddata").str.get(0)#
df_kdt_numbers['FileName'] = df_kdt_numbers['FileName'].str.split("-LR-5").str.get(0)

df_kdt_numbers['FileName'] = df_kdt_numbers['FileName'].str.split("-LR-5").str.get(0)
df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('CART', na=False)==True,'FileName'] = df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('CART', na=False)==True,'FileName'].str.split("_S").str.get(0)
df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('S00AF', na=False)==True,'FileName'] = df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('S00AF', na=False)==True,'FileName'].str.split("_S").str.get(0)
df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('MDA', na=False)==True,'FileName'] = df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('MDA', na=False)==True,'FileName'].str.split("_S").str.get(0)
df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('19696', na=False)==True,'FileName'] = df_kdt_numbers.loc[df_kdt_numbers['FileName'].str.startswith('19696', na=False)==True,'FileName'].str.split("_S").str.get(0)

df_kdt_numbers.index = df_kdt_numbers.FileName
common_names_kdt = set(df_kdt_numbers.index).intersection(set(df_CART_table_stool.stool_sample_id))

df_kdt_numbers_filtered = df_kdt_numbers.loc[common_names_kdt,:]

samples_below_thresholds = list(df_kdt_numbers_filtered.loc[df_kdt_numbers_filtered[' Number of reads kdt'] < 5e5].FileName)

# Patients exclusion:

df_CART_table_patients_filtered = df_CART_table_patients.loc[(df_CART_table_patients.subjectID).isin(df_CART_table_stool.subjectID) & # Excluding patinets due to lack of stool samples
                                                             ~(df_CART_table_patients.subjectID).isin(patient_mixed_basal_true) & # Excluding patinets due to mixed pre-infusing Abx. risks
                                                             ~df_CART_table_patients.Progression_early.isnull(),:] # Excluding patients due to an incomplete follow-up.

df_CART_table_patients_filtered.index = df_CART_table_patients_filtered.subjectID


# Identifying samples that do not belong to any included patient:

creteria_donor_included = df_CART_table_stool["subjectID"].isin(df_CART_table_patients_filtered.subjectID)

# Defining the criteria for samples inclusion:

creteria_stool_samples = creteria_donor_included & ~df_CART_table_stool.stool_sample_id.isin(samples_below_thresholds)


# Defining the final processed and filtered tables:

df_CART_table_stool_filtered = df_CART_table_stool.loc[creteria_stool_samples,:]
df_CART_table_taxa_filtered =  df_CART_table_taxa.loc[creteria_stool_samples,:]
df_CART_table_pathways_filtered = df_CART_table_pathways.loc[creteria_stool_samples,:]
df_CART_table_genes_filtered = df_CART_table_genes.loc[creteria_stool_samples,:]

df_CART_table_response_filtered = df_CART_table_response.loc[df_CART_table_response["subjectID"].isin(df_CART_table_patients_filtered.subjectID),:]
df_CART_table_blood_filtered = df_CART_table_blood.loc[df_CART_table_blood["subjectID"].isin(df_CART_table_patients_filtered.subjectID),:]


### Subsetting the samples based on the relative sampling day and the pre-infusion antibiotics-risk

In [ ]:
# "all":

df_CART_table_stool_all =    df_CART_table_stool_filtered.copy()
df_CART_table_taxa_all =     df_CART_table_taxa_filtered.copy()
df_CART_table_pathways_all = df_CART_table_pathways_filtered.copy()
df_CART_table_genes_all =    df_CART_table_genes_filtered.copy()


# "basal_all":

creteria_basal = (df_CART_table_stool_filtered["sampleDay_sample"] < 1) &  (df_CART_table_stool_filtered["sampleDay_sample"] > -22)

df_CART_table_stool_basal_all = df_CART_table_stool_filtered.loc[creteria_basal,:]
df_CART_table_taxa_basal_all = df_CART_table_taxa_filtered.loc[creteria_basal,:]
df_CART_table_pathways_basal_all = df_CART_table_pathways_filtered.loc[creteria_basal,:]
df_CART_table_genes_basal_all = df_CART_table_genes_filtered.loc[creteria_basal,:]


# "basal_noHighRiskAbx":

creteria_NoHighRisk = df_CART_table_stool_basal_all["Basal_no_HighRisk_sample"] == True

df_CART_table_stool_basal_NoHighRisk =    df_CART_table_stool_basal_all.loc[creteria_NoHighRisk,:]
df_CART_table_taxa_basal_NoHighRisk =     df_CART_table_taxa_basal_all.loc[creteria_NoHighRisk,:]
df_CART_table_pathways_basal_NoHighRisk = df_CART_table_pathways_basal_all.loc[creteria_NoHighRisk,:]
df_CART_table_genes_basal_NoHighRisk =    df_CART_table_genes_basal_all.loc[creteria_NoHighRisk,:]


### Deriving the mean-per patient basal microbiome

In [ ]:
def derive_mean_microbiome(microbiome_df, samples_df):
    
    _microbiome_df = microbiome_df.copy()
    _microbiome_df = _microbiome_df.div(_microbiome_df.sum(axis=1), axis=0)
    
    microbiome_df_mean = samples_df.loc[:, ['subjectID']].join(_microbiome_df).groupby("subjectID")[_microbiome_df.columns].mean()
    
    samples_df_mean = samples_df.groupby(samples_df["subjectID"]).apply(lambda _df : _df.iloc[0,:]) 
    samples_df_mean.index = samples_df_mean.subjectID

    return(microbiome_df_mean, samples_df_mean)


# basal_all_mean:

df_CART_table_stool_basal_all_mean =    derive_mean_microbiome(df_CART_table_taxa_basal_all, df_CART_table_stool_basal_all)[1]
df_CART_table_taxa_basal_all_mean =     derive_mean_microbiome(df_CART_table_taxa_basal_all, df_CART_table_stool_basal_all)[0]
df_CART_table_pathways_basal_all_mean = derive_mean_microbiome(df_CART_table_pathways_basal_all, df_CART_table_stool_basal_all)[0]
df_CART_table_genes_basal_all_mean =    derive_mean_microbiome(df_CART_table_genes_basal_all, df_CART_table_stool_basal_all)[0]


# basal_NoHighRisk_mean:

df_CART_table_stool_basal_NoHighRisk_mean = df_CART_table_stool_basal_all_mean.loc[df_CART_table_stool_basal_all_mean["Basal_no_HighRisk_sample"]]
df_CART_table_taxa_basal_NoHighRisk_mean = df_CART_table_taxa_basal_all_mean.loc[df_CART_table_stool_basal_all_mean["Basal_no_HighRisk_sample"]]
df_CART_table_pathways_basal_NoHighRisk_mean = df_CART_table_pathways_basal_all_mean.loc[df_CART_table_stool_basal_all_mean["Basal_no_HighRisk_sample"]]
df_CART_table_genes_basal_NoHighRisk_mean = df_CART_table_genes_basal_all_mean.loc[df_CART_table_stool_basal_all_mean["Basal_no_HighRisk_sample"]]



### Defining the function "abundance_filtration" and the abundance thresholds

In [ ]:
taxa_abundance_threshold = 1e-3
pathways_abundance_threshold = 1e-4
genes_abundance_threshold = 1e-4

def abundance_filtration(df_X, abundnace_threshold):
    
    _df_X = df_X.copy()
    _df_X = _df_X.div(_df_X.sum(axis=1), axis=0)

    _df_X[_df_X <= abundnace_threshold] = abundnace_threshold
    _df_X = _df_X.loc[:,_df_X.max() > abundnace_threshold]
    
    return _df_X.div(_df_X.sum(axis=1), axis=0)


### Calculating Shannon indices

In [ ]:
# Shannon for all samples: 

df_CART_table_taxa_all_norm = abundance_filtration(df_CART_table_taxa_all, taxa_abundance_threshold)

df_Shannon = pd.DataFrame(index=df_CART_table_taxa_all_norm.index, columns=["Shannon"])
                          
for current_sample in df_Shannon.index:
    df_Shannon.loc[current_sample, "Shannon"] = shannon(df_CART_table_taxa_all_norm.loc[current_sample,:])
    

# Shannon for the mean basal microbiome:

df_Shannon_basal_mean_all = pd.DataFrame(index=df_CART_table_taxa_basal_all_mean.index, columns=["Shannon"])

for current_subject in df_Shannon_basal_mean_all.index:
    df_Shannon_basal_mean_all.loc[current_subject, "Shannon"] = shannon(df_CART_table_taxa_basal_all_mean.loc[current_subject,:])


# Shannon for the NoHighRisk mean basal microbiome: 

df_Shannon_basal_mean_NoHighRisk = pd.DataFrame(index=df_CART_table_taxa_basal_NoHighRisk_mean.index, columns=["Shannon"])

for current_subject in df_Shannon_basal_mean_NoHighRisk.index:
    df_Shannon_basal_mean_NoHighRisk.loc[current_subject, "Shannon"] = shannon(df_CART_table_taxa_basal_NoHighRisk_mean.loc[current_subject,:])
    

### Defining stratifications and graphical-layouts for metadata parameters

In [ ]:
df_colvecs = pd.DataFrame(index=df_CART_table_stool_all.index)
df_colvecs["subjectID"] = df_CART_table_stool_all["subjectID"] 

# Patients parameters:

for current_patient in df_colvecs.subjectID:

    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Country"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Country"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Center"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Center"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Gender"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Gender"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Age"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Age"].values
    
    df_colvecs.loc[df_colvecs["subjectID"] == current_patient, "CR180"] = df_CART_table_response_filtered.loc[(df_CART_table_response_filtered["subjectID"] == current_patient) & (df_CART_table_response_filtered["Day"] == 180) ,"Response"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Progression_early"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Progression_early"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "CRS_grade"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CRS_grade"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "ICANS_grade"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"ICANS_grade"].values
  
    df_colvecs.loc[df_colvecs["subjectID"] == current_patient, "LDH_Depletion"] = df_CART_table_blood_filtered.loc[(df_CART_table_blood_filtered["subjectID"] == current_patient) & (df_CART_table_blood_filtered["Day"] == "Depletion") ,"LDH"].values
    df_colvecs.loc[df_colvecs["subjectID"] == current_patient, "CRP_depletion"] = df_CART_table_blood_filtered.loc[(df_CART_table_blood_filtered["subjectID"] == current_patient) & (df_CART_table_blood_filtered["Day"] == "Depletion") ,"CRP"].values
    df_colvecs.loc[df_colvecs["subjectID"] == current_patient, "IL6_depletion"] = df_CART_table_blood_filtered.loc[(df_CART_table_blood_filtered["subjectID"] == current_patient) & (df_CART_table_blood_filtered["Day"] == "Depletion") ,"IL6"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "CD3_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD3_mul_Aph"].values 
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "CD4_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD4_mul_Aph"].values 
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "CD8_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD8_mul_Aph"].values

    df_colvecs.loc[df_colvecs["subjectID"] == current_patient, "Product"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Product"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "ECOG_grade"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"ECOG"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "BulkyDisease_10cm"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"BulkyDisease_10cm"].values
    df_colvecs.loc[df_colvecs["subjectID"] ==current_patient, "Prior_Tx"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"Prior_Tx"].values
    
# Basal_no_HighRisk_sample:
df_colvecs["Basal_no_HighRisk_sample"] = df_CART_table_stool_all["Basal_no_HighRisk_sample"]
df_colvecs.loc[df_colvecs["Basal_no_HighRisk_sample"] == False, "Basal_no_HighRisk_sample_binarized"] = 0
df_colvecs.loc[df_colvecs["Basal_no_HighRisk_sample"] == True, "Basal_no_HighRisk_sample_binarized"] = 1

# AbxRisk_On_Sampling:
df_colvecs["AbxRisk_On_Sampling"] = df_CART_table_stool_all["AbxRisk_On_Sampling"]

# Shannon:
df_colvecs["Shannon"] = df_Shannon["Shannon"]

# Progression_early_binarized:
df_colvecs.loc[df_colvecs["Progression_early"] == "no", "Progression_early_binarized"] = 0
df_colvecs.loc[df_colvecs["Progression_early"] == "yes", "Progression_early_binarized"] = 1

# Age_binarized:
df_colvecs.loc[df_colvecs["Age"] < 65, "Age_binarized"] = 0
df_colvecs.loc[df_colvecs["Age"] >= 65, "Age_binarized"] = 1

# LDH_Depletion_binarized
LDH_threshold = 280
df_colvecs.loc[df_colvecs["LDH_Depletion"] < LDH_threshold, "LDH_Depletion_binarized"] = 0
df_colvecs.loc[df_colvecs["LDH_Depletion"] >= LDH_threshold, "LDH_Depletion_binarized"] = 1

# Response_d180_grade
df_colvecs.loc[df_colvecs["CR180"] == 'dead', "Response_d180_grade"] = 0
df_colvecs.loc[df_colvecs["CR180"] == 'PD', "Response_d180_grade"] = 1
df_colvecs.loc[df_colvecs["CR180"] == 'PR', "Response_d180_grade"] = 2
df_colvecs.loc[df_colvecs["CR180"] == 'CR', "Response_d180_grade"] = 3

# Response_binarized
df_colvecs.loc[df_colvecs["CR180"].isin(["PR", "PD", "dead"]), "CR180_binarized"] = 0
df_colvecs.loc[df_colvecs["CR180"] == "CR", "CR180_binarized"] = 1

# Survival
df_colvecs.loc[df_colvecs["CR180"] == "dead", "Survival_d180"] = 0
df_colvecs.loc[df_colvecs["CR180"].isin(["CR", "PR", "PD"]), "Survival_d180"] = 1

# CRS_01 binarized
df_colvecs.loc[df_colvecs.CRS_grade < 2 , "CRS_01"] = 1
df_colvecs.loc[df_colvecs.CRS_grade > 1 , "CRS_01"] = 0

# CRS 0, 1, 2, 3+4 categorized
df_colvecs.loc[df_colvecs.CRS_grade == 0 , "CRS_012_34"] = 0
df_colvecs.loc[df_colvecs.CRS_grade == 1 , "CRS_012_34"] = 1
df_colvecs.loc[df_colvecs.CRS_grade == 2 , "CRS_012_34"] = 2
df_colvecs.loc[df_colvecs.CRS_grade == 3 , "CRS_012_34"] = 34
df_colvecs.loc[df_colvecs.CRS_grade == 4 , "CRS_012_34"] = 34

# ICANS_012_34:
df_colvecs.loc[df_colvecs.ICANS_grade == 0 , "ICANS_012_34"] = 0
df_colvecs.loc[df_colvecs.ICANS_grade == 1 , "ICANS_012_34"] = 1
df_colvecs.loc[df_colvecs.ICANS_grade == 2 , "ICANS_012_34"] = 2
df_colvecs.loc[df_colvecs.ICANS_grade == 3 , "ICANS_012_34"] = 34
df_colvecs.loc[df_colvecs.ICANS_grade == 4 , "ICANS_012_34"] = 34

# ECOG_01, ECOG_01_colvec:
df_colvecs["ECOG_01_colvec"] = "gray"
df_colvecs.loc[df_colvecs.ECOG_grade < 2 , "ECOG_01"] = 1
df_colvecs.loc[df_colvecs.ECOG_grade < 2 , "ECOG_01_colvec"] = "green"
df_colvecs.loc[df_colvecs.ECOG_grade > 1 , "ECOG_01"] = 0
df_colvecs.loc[df_colvecs.ECOG_grade > 1 , "ECOG_01_colvec"] = "red"
colvec_label_ECOG_01   = ['ECOG 0-1','ECOG 2-3']
colvec_color_ECOG_01  = ["green", "red"]
colvec_marker_ECOG_01    = ["o","o","o"]

# country_colvec, Country_yesno:
df_colvecs["country_colvec"] = "gray"
df_colvecs.loc[df_colvecs.Country == "Germany","country_colvec"] = "green"
df_colvecs.loc[df_colvecs.Country == "US","country_colvec"] = "red"
df_colvecs.loc[df_colvecs.Country == "Germany","Country_yesno"] = "yes"
df_colvecs.loc[df_colvecs.Country == "US","Country_yesno"] = "no"
colvec_label_country   = ['Germany','US']
colvec_color_country  = ["green", "red"]
colvec_marker_country    = ["o","o"]

# AbxRisk_On_Sampling_colvec:
df_colvecs["AbxRisk_On_Sampling_colvec"] = "gray"
df_colvecs.loc[df_colvecs.AbxRisk_On_Sampling == "HR","AbxRisk_On_Sampling_colvec"] = "orange"
df_colvecs.loc[df_colvecs.AbxRisk_On_Sampling == "LR","AbxRisk_On_Sampling_colvec"] = "lightgreen"
df_colvecs.loc[df_colvecs.AbxRisk_On_Sampling == "None","AbxRisk_On_Sampling_colvec"] = "steelblue"
colvec_label_AbxRisk_On_Sampling   = ['HR','LR', 'None']
colvec_color_AbxRisk_On_Sampling  = ["orange", "lightgreen", "steelblue"]
colvec_marker_AbxRisk_On_Sampling    = ["o","o","o"]

# day_colvec:
df_colvecs["day_colvec"] = df_CART_table_stool_all.sampleDay_sample
cmap_day = mpl.cm.Spectral
norm_day = mpl.colors.Normalize(vmin=-21, vmax=21)

# Prior_Tx_binned, Prior_Tx_binned_colvec
df_colvecs.loc[(df_colvecs["Prior_Tx"] < 4) & (df_colvecs["Prior_Tx"] >= 0), "Prior_Tx_binned"] = 1
df_colvecs.loc[(df_colvecs["Prior_Tx"] > 3) & (df_colvecs["Prior_Tx"] < 7), "Prior_Tx_binned"] = 2
df_colvecs.loc[(df_colvecs["Prior_Tx"] > 6) & (df_colvecs["Prior_Tx"] <= 10), "Prior_Tx_binned"] = 3
df_colvecs["Prior_Tx_binned_colvec"] = "black"
df_colvecs.loc[df_colvecs.Prior_Tx_binned == 1,"Prior_Tx_binned_colvec"] = "deepskyblue"
df_colvecs.loc[df_colvecs.Prior_Tx_binned == 2,"Prior_Tx_binned_colvec"] = "gray"
df_colvecs.loc[df_colvecs.Prior_Tx_binned == 3,"Prior_Tx_binned_colvec"] = "red"
colvec_label_Prior_Tx_binned   = ['0-3','4-6', '7-10', 'Not reported']
colvec_color_Prior_Tx_binned  = ["deepskyblue", "gray","red", "black"]
colvec_marker_Prior_Tx_binned    = ["o","o","o", "o"]

# CR+none-LR / nonCR+non-LR / CR+HR / nonCR+HR :
df_colvecs.loc[(df_colvecs["CR180_binarized"] == 1 ) & (df_colvecs["Basal_no_HighRisk_sample_binarized"] == 1), "CR180_noHRAbx"] = "noHR + CR"
df_colvecs.loc[(df_colvecs["CR180_binarized"] == 0 ) & (df_colvecs["Basal_no_HighRisk_sample_binarized"] == 1), "CR180_noHRAbx"] = "noHR + noCR"
df_colvecs.loc[(df_colvecs["CR180_binarized"] == 1 ) & (df_colvecs["Basal_no_HighRisk_sample_binarized"] == 0), "CR180_noHRAbx"] = "HR + CR"
df_colvecs.loc[(df_colvecs["CR180_binarized"] == 0 ) & (df_colvecs["Basal_no_HighRisk_sample_binarized"] == 0), "CR180_noHRAbx"] = "HR + noCR"

# Subsettting df_colvecs:
df_colvecs_all = df_colvecs.copy()

df_CART_table_stool_basal_all_mean.index = df_CART_table_stool_basal_all_mean.stool_sample_id
df_CART_table_stool_basal_NoHighRisk_mean.index = df_CART_table_stool_basal_NoHighRisk_mean.stool_sample_id

df_colvecs_basal_all_mean = df_colvecs_all.loc[df_CART_table_stool_basal_all_mean.index,:]
df_colvecs_basal_NoHighRisk_mean = df_colvecs.loc[ df_CART_table_stool_basal_NoHighRisk_mean.index,:]

df_colvecs_basal_all_mean.index = df_CART_table_stool_basal_all_mean.subjectID
df_colvecs_basal_NoHighRisk_mean.index = df_CART_table_stool_basal_NoHighRisk_mean.subjectID



### Beta-diversity PCoA analyses
* Figure 2E (left and right) 
* Extended Data Figures 5E (left and right), 5F, 5G and 5H

In [ ]:
class MarkerHandler(HandlerBase):
    def create_artists(self, legend, tup,xdescent, ydescent,
                        width, height, fontsize,trans):
        return [plt.Line2D([width/2], [height/2.],ls="",
                       marker=tup[1],color=tup[0], transform=trans)]
    

def plot_dr(df_X_raw, abundance_threshold, colvec, list_color = None ,list_mak = None, list_lab = None, title_name = None, cmap = None, norm = None, cbar = False):
    
    df_X = abundance_filtration(df_X_raw, abundance_threshold)
    curr_figsize = [8,8]
    
    df_X_dist = distance.squareform(distance.pdist(df_X, metric="braycurtis"))
    df_X_pcoa = pcoa(df_X_dist)
    pcoa_explained_var = np.ndarray.round(np.array(df_X_pcoa.proportion_explained[["PC1", "PC2"]] * 100) ,1)

    plt.figure(figsize = curr_figsize)
    
    pcoa_1 = np.array(df_X_pcoa.samples['PC1'])
    pcoa_2 = np.array(df_X_pcoa.samples['PC2'])
    
    pcoa_1 = pcoa_1[~pd.isna( colvec)]
    pcoa_2 = pcoa_2[~pd.isna( colvec)]
    colvec = colvec[~pd.isna( colvec)]
    
    plt.scatter(pcoa_1, pcoa_2, c = colvec, cmap = cmap, norm = norm)
    plt.title("PCoA_"+ title_name,fontsize=16)
    plt.xlabel('PCoA axis 1 ['+ np.array2string(pcoa_explained_var[0])+'%]',fontsize = 16)
    plt.ylabel('PCoA axis 2 ['+ np.array2string(pcoa_explained_var[1])+'%]',fontsize = 16)
    
    if cbar == False:
        plt.legend(list(zip(list_color,list_mak)), list_lab, handler_map={tuple:MarkerHandler()}) 
    
    if cbar == True:   
        plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap))
        
    plt.savefig(path_output + title_name + '.pdf')
    plt.close()
    
    return


# Fig_2E (left and right)
plot_dr(df_CART_table_taxa_all, taxa_abundance_threshold,  df_colvecs_all["AbxRisk_On_Sampling_colvec"], colvec_color_AbxRisk_On_Sampling, colvec_marker_AbxRisk_On_Sampling, colvec_label_AbxRisk_On_Sampling, "CART_Microbiome_Fig_2E_left_taxa")
plot_dr(df_CART_table_pathways_all, pathways_abundance_threshold,  df_colvecs_all["AbxRisk_On_Sampling_colvec"], colvec_color_AbxRisk_On_Sampling, colvec_marker_AbxRisk_On_Sampling, colvec_label_AbxRisk_On_Sampling, "CART_Microbiome_Fig_2E_right_pathwayds")

# Fig_S5E (left and right)
plot_dr(df_CART_table_taxa_all, taxa_abundance_threshold,  df_colvecs_all["country_colvec"], colvec_color_country, colvec_marker_country, colvec_label_country , "CART_Microbiome_Fig_S5E_left_taxa")
plot_dr(df_CART_table_pathways_all, pathways_abundance_threshold,  df_colvecs_all["country_colvec"], colvec_color_country, colvec_marker_country, colvec_label_country , "CART_Microbiome_Fig_S5E_right_pathways")

# Fig_S5F
plot_dr(df_CART_table_taxa_all, taxa_abundance_threshold,  df_colvecs_all["ECOG_01_colvec"], colvec_color_ECOG_01, colvec_marker_ECOG_01, colvec_label_ECOG_01 , "CART_Microbiome_Fig_S5F")

# Fig_S5G
plot_dr(df_CART_table_taxa_all.loc[~np.isnan(df_colvecs_all["Prior_Tx_binned"])], taxa_abundance_threshold,  df_colvecs_all.loc[~np.isnan(df_colvecs_all["Prior_Tx_binned"]), "Prior_Tx_binned_colvec"], colvec_color_Prior_Tx_binned[0:3], colvec_marker_Prior_Tx_binned[0:3], colvec_label_Prior_Tx_binned[0:3] , "CART_Microbiome_Fig_S5G")

# Fig_S5H
plot_dr(df_CART_table_taxa_all, taxa_abundance_threshold,  df_colvecs_all["day_colvec"], title_name= "CART_Microbiome_Fig_S5H", cmap = cmap_day, norm = norm_day, cbar=True)


### Differential-abundance analyses
* Figure 2F
* Extended Figures 6A, 6B, 6C, 6D, 7A, 7B, 7C and 7D (volcano plots and corresponding log2 fold-change bar-plots)

In [ ]:
# ------------------------------------------------------------------------
# Differential abundance analyses part 1: Defining the functions
# ------------------------------------------------------------------------

def differential_abundance_analysis(df_X_raw, abundance_threshold, df_y, split_label):
  
    df_X = abundance_filtration(df_X_raw, abundance_threshold)
  
    mannwhitneyu_results_df = pd.DataFrame(index = df_X.columns)
    
    df_X_no = df_X.loc[(df_y[split_label] == 0) | (df_y[split_label] == "no") ,:]
    df_X_yes = df_X.loc[(df_y[split_label] == 1) | (df_y[split_label] == "yes"),:]
    
    for current_feature in df_X.columns:
        current_feature_no = df_X_no[current_feature]
        current_feature_yes = df_X_yes[current_feature]
            
        statistic, pvalue = mannwhitneyu(current_feature_no,current_feature_yes,alternative='two-sided')
        
        mannwhitneyu_results_df.loc[current_feature, "log2_fold_change"] = np.log2(current_feature_yes.mean() / current_feature_no.mean())
        mannwhitneyu_results_df.loc[current_feature, "Pval"] = pvalue
            
    padj_binary, padj =  fdrcorrection(mannwhitneyu_results_df.Pval, alpha=0.2, method = 'i')
    mannwhitneyu_results_df.loc[:, "Padjusted"] = padj
    
    return(mannwhitneyu_results_df)


def plot_volcano(mannwhitneyu_res_df, out_data_path, title_name, p_thr,  label_yes = "Yes", label_no = "No"):
    
    fold_change_threshold = 2
    
    feature_names = list(mannwhitneyu_res_df.index)
    log_fold_change = mannwhitneyu_res_df["log2_fold_change"]
    p_value = mannwhitneyu_res_df["Pval"]
    
    hits_index = (p_value < p_thr) & (abs(log_fold_change) > np.log2(fold_change_threshold))
    
    # The volcano plot:
    mpl.rcParams.update(mpl.rcParamsDefault)
    plt.rcParams["font.size"] = "6"
    plt.figure(figsize=[10,10])
    plt.scatter(log_fold_change, -np.log10(p_value),c='gray',s=8,label='Other features')
    plt.scatter(log_fold_change[hits_index],-np.log10(p_value[hits_index]),c='red',s = 30, label='Hit features')
    plt.xlabel('Log2 fold-change ('+ label_yes + " / " + label_no +" )",fontsize = 16)
    plt.ylabel('-Log10 Pval',fontsize = 16)
    plt.ylim(0,max(-np.log10(p_value) + 0.1))
    plt.xlim(-max(abs(log_fold_change)) -0.2 ,max(abs(log_fold_change)) + 0.2)
    plt.title(title_name,fontsize=16)
    plt.plot([0,0], [0, max(-np.log10(p_value)) + 0.1], c = "black")
    plt.plot([-np.log2(fold_change_threshold), -np.log2(fold_change_threshold)], [0, max(-np.log10(p_value)) + 0.1], '--',c = "gray")
    plt.plot([np.log2(fold_change_threshold), np.log2(fold_change_threshold)], [0, max(-np.log10(p_value)) + 0.1], '--',c = "gray")
    plt.plot([-max(abs(log_fold_change)) -0.2, max(abs(log_fold_change)) + 0.2], [-np.log10(p_thr), -np.log10(p_thr)], '--',c = "gray")
    plt.legend(loc=(1.03,0.8),fontsize=14,fancybox=True)
    plt.savefig(path_output + title_name + '_volcano.pdf',dpi=300, bbox_inches='tight')
    plt.close()
 
    # The associated bar-plot:
    
    mannwhitneyu_res_df_significant = mannwhitneyu_res_df.loc[(mannwhitneyu_res_df.Pval < p_thr) & 
                                                                                 (abs(mannwhitneyu_res_df.log2_fold_change) > np.log2(fold_change_threshold)),:]
    if len(hits_index) > -1:

        mannwhitneyu_res_df_significant_sorted = mannwhitneyu_res_df_significant.sort_values(by = "log2_fold_change")
        
        plt.figure(figsize=[10,20])
        plt.barh(mannwhitneyu_res_df_significant_sorted.index, mannwhitneyu_res_df_significant_sorted.log2_fold_change)
        
        plt.xlabel('Log2 fold-change ('+ label_yes + " / " + label_no +" )",fontsize = 16)
        plt.tick_params(axis='both', which='major', labelsize=10)
        plt.savefig(path_output + title_name + '_bars.pdf',dpi=300, bbox_inches='tight')
        plt.close()
    
    return


def derive_all_differential_abundance_results(microbiome_df, microbiome_thr, col_vec_df,  subset_name, stratifeing_parameter, figfilename):
    
    df_res_dabund = differential_abundance_analysis(microbiome_df, microbiome_thr, col_vec_df, stratifeing_parameter)
    combined_name = "_" + subset_name + "_" + stratifeing_parameter
    plot_volcano(df_res_dabund, path_output, figfilename, p_thr = 0.05)

    return(df_res_dabund)
    

# ------------------------------------------------------------------------
# Differential abundance analyses part 2: Performing the analyses
# ------------------------------------------------------------------------

# Fig_2F
df_res_dabund_basal_all_Basal_no_HighRisk_sample_taxa= \
        derive_all_differential_abundance_results( \
        df_CART_table_taxa_basal_all_mean,\
        taxa_abundance_threshold, df_colvecs_basal_all_mean, \
        "Basal_all", "Basal_no_HighRisk_sample", "CART_Microbiome_Fig_2F") 


# Fig_S6AB
df_res_dabund_basal_all_Basal_no_HighRisk_sample_pathways = \
        derive_all_differential_abundance_results( \
        df_CART_table_pathways_basal_all_mean, \
        pathways_abundance_threshold, df_colvecs_basal_all_mean, \
        "Basal_all", "Basal_no_HighRisk_sample", "CART_Microbiome_Fig_S6AB_pathways") 


# Fig_S6CD (volcano) and S6D (bar plot) taxa
df_res_dabund_basal_all_Country_taxa = \
        derive_all_differential_abundance_results( \
        df_CART_table_taxa_basal_all_mean,\
        taxa_abundance_threshold, df_colvecs_basal_all_mean, \
        "Basal_all", "Country_yesno", "CART_Microbiome_Fig_S6CD_taxa" ) 


# Fig_S6CD (volcano) and S6D (bar plot) pathways
df_res_dabund_basal_all_Country_pathways = \
        derive_all_differential_abundance_results( \
        df_CART_table_pathways_basal_all_mean, \
        pathways_abundance_threshold, df_colvecs_basal_all_mean, \
        "Basal_all", "Country_yesno", "CART_Microbiome_Fig_S6CD_pathways" ) 


# Fig_S7A
df_res_dabund_basal_all_Survival_taxa= \
        derive_all_differential_abundance_results( \
        df_CART_table_taxa_basal_all_mean, \
        taxa_abundance_threshold,  df_colvecs_basal_all_mean, \
        "Basal_all", "Survival_d180", "CART_Microbiome_Fig_S7A")


# Fig_S7B
df_res_dabund_basal_NoHighRisk_Survival_taxa = \
        derive_all_differential_abundance_results( \
        df_CART_table_taxa_basal_NoHighRisk_mean, \
        taxa_abundance_threshold, df_colvecs_basal_NoHighRisk_mean, \
        "Basal_NoHighRisk", "Survival_d180", "CART_Microbiome_Fig_S7B")


# Fig_S7C
df_res_dabund_basal_NoHighRisk_Survival_pathways = \
        derive_all_differential_abundance_results( \
        df_CART_table_pathways_basal_NoHighRisk_mean, \
        pathways_abundance_threshold, df_colvecs_basal_NoHighRisk_mean, \
        "Basal_NoHighRisk", "Survival_d180", "CART_Microbiome_Fig_S7C")


# Fig_S7D
df_res_dabund_basal_NoHighRisk_Progression_early_pathways = \
        derive_all_differential_abundance_results( \
        df_CART_table_pathways_basal_NoHighRisk_mean, \
        pathways_abundance_threshold, df_colvecs_basal_all_mean, \
        "Basal_NoHighRisk", "Progression_early", "CART_Microbiome_Fig_S7D") 



### PERMANOVA analyses
* Figures 2D, 2G, 2H

In [ ]:
# ------------------------------------------------------------------------
# PERMANOVA part 1: Performing the PERMANOVA analyses
# ------------------------------------------------------------------------

def analyze_permanova(df_X, grouping_vec, abundance_threshold):
    np.random.seed(0)

    df_X_filtered = abundance_filtration(df_X, abundance_threshold)
    
    df_X_filtered_noNaN = df_X_filtered.loc[pd.isna(grouping_vec) == False,:]
    grouping_vec_noNaN = grouping_vec[pd.isna(grouping_vec) == False]
    
    grouping_vec_noNaN = grouping_vec_noNaN.astype(str)
    
    dm_X = DistanceMatrix(distance.squareform(distance.pdist(df_X_filtered_noNaN, metric='braycurtis'))) # PERMANOVA based on braycurtis (addded on 06 June 2022)

    permanova_results = permanova(dm_X, grouping_vec_noNaN)

    r2 = 1 - 1 / (1 + permanova_results[4] * permanova_results[3] / (permanova_results[2] - permanova_results[3] - 1))
    permanova_results["R2"] = r2
    
    return(permanova_results)


permanova_cateogry_names = ["Response (day 180)", \
                            "Survival (day 180)", \
                            "Early progression", \
                            "CRS > 1", \
                            "ICANS grades",\
                            "ECOG > 1", \
                            "no_HighRisk_sample", \
                            "Country", \
                            "Gender", \
                            "Age >= 65", \
                            "LDH depletion >= 280", \
                            "Bulky disease 10cm", \
                            "Prior therapies 0-3 / 4-6 / 7-10",
                            "NHR_Response",
                            "NHS_Survival",
                            "NHR_Progression",
                            "NHR_CRS",
                            "NHR_ICANS"]


# PERMANOVA analyses based on all the basal samples:
# --------------------------------------------------

# Criterion: Response (day 180) (i.e. Complete remission, CR, at day 180)
permanova_taxa_basal_CR180 = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.CR180_binarized, taxa_abundance_threshold)
permanova_pathways_basal_CR180 = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.CR180_binarized, pathways_abundance_threshold)
permanova_genes_basal_CR180 = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.CR180_binarized, genes_abundance_threshold)

# Criterion: Survival (day 180)
permanova_taxa_basal_Survival = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Survival_d180, taxa_abundance_threshold)
permanova_pathways_basal_Survival = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Survival_d180, pathways_abundance_threshold)
permanova_genes_basal_Survival = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Survival_d180, genes_abundance_threshold)

# Criterion: Early progression
permanova_taxa_basal_Progression_early = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Progression_early_binarized, taxa_abundance_threshold)
permanova_pathways_basal_Progression_early = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Progression_early_binarized, pathways_abundance_threshold)
permanova_genes_basal_Progression_early = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Progression_early_binarized, genes_abundance_threshold)

# Criterion: CRS > 1
permanova_taxa_basal_CRS_01 = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.CRS_01, taxa_abundance_threshold)
permanova_pathways_basal_CRS_01 = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.CRS_01, pathways_abundance_threshold)
permanova_genes_basal_CRS_01 = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.CRS_01, genes_abundance_threshold)

# Criterion: ICANS grades
permanova_taxa_basal_ICANS_grade = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.ICANS_grade, taxa_abundance_threshold)
permanova_pathways_basal_ICANS_grade = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.ICANS_grade, pathways_abundance_threshold)
permanova_genes_basal_ICANS_grade = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.ICANS_grade, genes_abundance_threshold)

# Criterion: ECOG > 1
permanova_taxa_basal_ECOG_01 = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.ECOG_01, taxa_abundance_threshold)
permanova_pathways_basal_ECOG_01 = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.ECOG_01, pathways_abundance_threshold)
permanova_genes_basal_ECOG_01 = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.ECOG_01, genes_abundance_threshold)

# Criterion: no_HighRisk_sample
permanova_taxa_basal_no_HighRisk_sample = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Basal_no_HighRisk_sample, taxa_abundance_threshold)
permanova_pathways_basal_no_HighRisk_sample = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Basal_no_HighRisk_sample, pathways_abundance_threshold)
permanova_genes_basal_no_HighRisk_sample = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Basal_no_HighRisk_sample, genes_abundance_threshold)

# Criterion: Country
permanova_taxa_basal_Country = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Country, taxa_abundance_threshold)
permanova_pathways_basal_Country = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Country, pathways_abundance_threshold)
permanova_genes_basal_Country = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Country, genes_abundance_threshold)

# Criterion: Gender
permanova_taxa_basal_Gender = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Gender, taxa_abundance_threshold)
permanova_pathways_basal_Gender = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Gender, pathways_abundance_threshold)
permanova_genes_basal_Gender = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Gender, genes_abundance_threshold)

# Criterion: Age >= 65
permanova_taxa_basal_Age = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Age_binarized, taxa_abundance_threshold)
permanova_pathways_basal_Age = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Age_binarized, pathways_abundance_threshold)
permanova_genes_basal_Age = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Age_binarized, genes_abundance_threshold)

# Criterion: LDH depletion >= 280
permanova_taxa_basal_LDH_Depletion = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.LDH_Depletion_binarized, taxa_abundance_threshold)
permanova_pathways_basal_LDH_Depletion = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.LDH_Depletion_binarized, pathways_abundance_threshold)
permanova_genes_basal_LDH_Depletion = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.LDH_Depletion_binarized, genes_abundance_threshold)

# Criterion: BulkyDisease_10cm
permanova_taxa_basal_BulkyDisease = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.BulkyDisease_10cm, taxa_abundance_threshold)
permanova_pathways_basal_BulkyDisease = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.BulkyDisease_10cm, pathways_abundance_threshold)
permanova_genes_basal_BulkyDisease = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.BulkyDisease_10cm, genes_abundance_threshold)

# Criterion: Prior therapies 0-3 / 4-6 / 7-10
permanova_taxa_basal_Prior_Tx_binned = analyze_permanova(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean.Prior_Tx_binned, taxa_abundance_threshold)
permanova_pathways_basal_Prior_Tx_binned = analyze_permanova(df_CART_table_pathways_basal_all_mean, df_colvecs_basal_all_mean.Prior_Tx_binned, pathways_abundance_threshold)
permanova_genes_basal_Prior_Tx_binned = analyze_permanova(df_CART_table_genes_basal_all_mean, df_colvecs_basal_all_mean.Prior_Tx_binned, genes_abundance_threshold)


# PERMANOVA analyses based on the no_HighRisk basal samples:
# ----------------------------------------------------------

# Criterion: Response (day 180) (i.e. Complete remission, CR, at day 180)
permanova_taxa_basal_NoHighRisk_CR180 = analyze_permanova(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CR180_binarized, taxa_abundance_threshold)
permanova_pathways_basal_NoHighRisk_CR180 = analyze_permanova(df_CART_table_pathways_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CR180_binarized, pathways_abundance_threshold)
permanova_genes_basal_NoHighRisk_CR180 = analyze_permanova(df_CART_table_genes_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CR180_binarized, genes_abundance_threshold)

# Criterion: Survival (day 180)
permanova_taxa_basal_NoHighRisk_Survival = analyze_permanova(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Survival_d180, taxa_abundance_threshold)
permanova_pathways_basal_NoHighRisk_Survival = analyze_permanova(df_CART_table_pathways_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Survival_d180, pathways_abundance_threshold)
permanova_genes_basal_NoHighRisk_Survival = analyze_permanova(df_CART_table_genes_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Survival_d180, genes_abundance_threshold)

# Criterion: Early progression
permanova_taxa_basal_NoHighRisk_Progression_early = analyze_permanova(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Progression_early_binarized, taxa_abundance_threshold)
permanova_pathways_basal_NoHighRisk_Progression_early = analyze_permanova(df_CART_table_pathways_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Progression_early_binarized, pathways_abundance_threshold)
permanova_genes_basal_NoHighRisk_Progression_early = analyze_permanova(df_CART_table_genes_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.Progression_early_binarized, genes_abundance_threshold)

# Criterion: CRS > 1
permanova_taxa_basal_NoHighRisk_CRS_01 = analyze_permanova(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CRS_01, taxa_abundance_threshold)
permanova_pathways_basal_NoHighRisk_CRS_01 = analyze_permanova(df_CART_table_pathways_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CRS_01, pathways_abundance_threshold)
permanova_genes_basal_NoHighRisk_CRS_01 = analyze_permanova(df_CART_table_genes_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.CRS_01, genes_abundance_threshold)

# Criterion: ICANS grades
permanova_taxa_basal_NoHighRisk_ICANS_grade = analyze_permanova(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.ICANS_grade, taxa_abundance_threshold)
permanova_pathways_basal_NoHighRisk_ICANS_grade = analyze_permanova(df_CART_table_pathways_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.ICANS_grade, pathways_abundance_threshold)
permanova_genes_basal_NoHighRisk_ICANS_grade = analyze_permanova(df_CART_table_genes_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean.ICANS_grade, genes_abundance_threshold)

# ------------------------------------------------------------------------
# PERMANOVA part 2: Organizing the PERMANOVA results as heatmaps
# ------------------------------------------------------------------------

# Taxa permanova results:

permanova_taxa_list =[permanova_taxa_basal_CR180,
                      permanova_taxa_basal_Survival,
                      permanova_taxa_basal_Progression_early,
                      permanova_taxa_basal_CRS_01,
                      permanova_taxa_basal_ICANS_grade,
                      permanova_taxa_basal_ECOG_01,
                      permanova_taxa_basal_no_HighRisk_sample,
                      permanova_taxa_basal_Country,
                      permanova_taxa_basal_Gender,
                      permanova_taxa_basal_Age,
                      permanova_taxa_basal_LDH_Depletion,
                      permanova_taxa_basal_BulkyDisease,
                      permanova_taxa_basal_Prior_Tx_binned,
                      permanova_taxa_basal_NoHighRisk_CR180,
                      permanova_taxa_basal_NoHighRisk_Survival,
                      permanova_taxa_basal_NoHighRisk_Progression_early,
                      permanova_taxa_basal_NoHighRisk_CRS_01,
                      permanova_taxa_basal_NoHighRisk_ICANS_grade]

permanova_taxa_basal_all =  pd.concat(permanova_taxa_list, axis=1)
permanova_taxa_basal_all.columns = permanova_cateogry_names


# Pathways permanova results:

permanova_pathways_list =[permanova_pathways_basal_CR180,
                      permanova_pathways_basal_Survival,
                      permanova_pathways_basal_Progression_early,
                      permanova_pathways_basal_CRS_01,
                      permanova_pathways_basal_ICANS_grade,
                      permanova_pathways_basal_ECOG_01,
                      permanova_pathways_basal_no_HighRisk_sample,
                      permanova_pathways_basal_Country,
                      permanova_pathways_basal_Gender,
                      permanova_pathways_basal_Age,
                      permanova_pathways_basal_LDH_Depletion,
                      permanova_pathways_basal_BulkyDisease,
                      permanova_pathways_basal_Prior_Tx_binned,
                      permanova_pathways_basal_NoHighRisk_CR180,
                      permanova_pathways_basal_NoHighRisk_Survival,
                      permanova_pathways_basal_NoHighRisk_Progression_early,
                      permanova_pathways_basal_NoHighRisk_CRS_01,
                      permanova_pathways_basal_NoHighRisk_ICANS_grade]

permanova_pathways_basal_all =  pd.concat(permanova_pathways_list, axis=1)
permanova_pathways_basal_all.columns = permanova_cateogry_names


# Genes permanova results:

permanova_genes_list =[permanova_genes_basal_CR180,
                      permanova_genes_basal_Survival,
                      permanova_genes_basal_Progression_early,
                      permanova_genes_basal_CRS_01,
                      permanova_genes_basal_ICANS_grade,
                      permanova_genes_basal_ECOG_01,
                      permanova_genes_basal_no_HighRisk_sample,
                      permanova_genes_basal_Country,
                      permanova_genes_basal_Gender,
                      permanova_genes_basal_Age,
                      permanova_genes_basal_LDH_Depletion,
                      permanova_genes_basal_BulkyDisease,
                      permanova_genes_basal_Prior_Tx_binned,
                      permanova_genes_basal_NoHighRisk_CR180,
                      permanova_genes_basal_NoHighRisk_Survival,
                      permanova_genes_basal_NoHighRisk_Progression_early,
                      permanova_genes_basal_NoHighRisk_CRS_01,
                      permanova_genes_basal_NoHighRisk_ICANS_grade]


permanova_genes_basal_all =  pd.concat(permanova_genes_list, axis=1)
permanova_genes_basal_all.columns = permanova_cateogry_names

permanova_basal_summary_pval = pd.DataFrame(index = permanova_taxa_basal_all.columns, columns=["Species", "Pathways", "Genes"] )
permanova_basal_summary_pval.loc[:,"Species"] = permanova_taxa_basal_all.loc["p-value",:]
permanova_basal_summary_pval.loc[:,"Pathways"] = permanova_pathways_basal_all.loc["p-value",:]
permanova_basal_summary_pval.loc[:,"Genes"] = permanova_genes_basal_all.loc["p-value",:]

permanova_basal_summary_pval_symbol = permanova_basal_summary_pval.copy()

permanova_basal_summary_pval_symbol[permanova_basal_summary_pval >= 0.05] = ""
permanova_basal_summary_pval_symbol[permanova_basal_summary_pval < 0.05] = "\n*"
permanova_basal_summary_pval_symbol[permanova_basal_summary_pval < 0.01] = "\n**"
permanova_basal_summary_pval_symbol[permanova_basal_summary_pval < 0.001] = "\n***"

permanova_basal_summary_R2 = pd.DataFrame(index = permanova_taxa_basal_all.columns, columns=["Species", "Pathways", "Genes"] )
permanova_basal_summary_R2.loc[:,"Species"] = permanova_taxa_basal_all.loc["R2",:]
permanova_basal_summary_R2.loc[:,"Pathways"] = permanova_pathways_basal_all.loc["R2",:]
permanova_basal_summary_R2.loc[:,"Genes"] = permanova_genes_basal_all.loc["R2",:]
permanova_basal_summary_R2_prercentage = round((permanova_basal_summary_R2.astype("float") * 100),1)


# Fig_2D:

permanova_table_1_categogeries = ["Age >= 65", "Gender",  "Country", "ECOG > 1", "LDH depletion >= 280", "Bulky disease 10cm", "Prior therapies 0-3 / 4-6 / 7-10", "no_HighRisk_sample"]
permanova_table_1_category_lables = ["Age", "Gender", "Country", "ECOG", "LDH", "Bulky disease", "Prior therapies", "Antibiotics"]

permanova_table_1_R2 = permanova_basal_summary_R2_prercentage.loc[permanova_table_1_categogeries,:]
permanova_table_1_R2.index = permanova_table_1_category_lables
permanova_table_1_pval_symbols = permanova_basal_summary_pval_symbol.loc[permanova_table_1_categogeries,:]
permanova_table_1_labels =  permanova_table_1_R2.values.astype(str) + np.asarray(permanova_table_1_pval_symbols) 

plt.figure(figsize = (8,8))
sns.set(font_scale=1.5)
annot_kws = {"verticalalignment": 'center_baseline'}
permanova_basa_plot_1 = sns.heatmap(permanova_table_1_R2, fmt="", cmap='YlGnBu', annot = permanova_table_1_labels, linewidths=2, cbar_kws={'label': 'Variance explained (%)'}, annot_kws=annot_kws)
permanova_basa_plot_1.set_xticklabels(permanova_basa_plot_1.get_xticklabels(), rotation=45, rotation_mode='anchor', ha='right')
permanova_basa_plot_1 = permanova_basa_plot_1.get_figure()
plt.tight_layout()
permanova_basa_plot_1.savefig(path_output + 'CART_Microbiome_Fig_2D.pdf')
plt.close()


# Fig_2G:

permanova_table_2_categogeries = ["Response (day 180)", "Survival (day 180)", "Early progression", "CRS > 1", "ICANS grades"]
permanova_table_2_3_category_lables = ["Response", "Survival", "Early progres.", "CRS", "ICANS"]

permanova_table_2_R2 = permanova_basal_summary_R2_prercentage.loc[permanova_table_2_categogeries,:]
permanova_table_2_R2.index = permanova_table_2_3_category_lables
permanova_table_2_pval_symbols = permanova_basal_summary_pval_symbol.loc[permanova_table_2_categogeries,:]
permanova_table_2_labels =  permanova_table_2_R2.values.astype(str) + np.asarray(permanova_table_2_pval_symbols) 
plt.close()

plt.figure(figsize = (8,6))
sns.set(font_scale=1.5)
permanova_basa_plot_2 = sns.heatmap(permanova_table_2_R2, fmt="", cmap='YlGnBu', annot = permanova_table_2_labels, linewidths=2, cbar_kws={'label': 'Variance explained (%)'})
permanova_basa_plot_2.set_xticklabels(permanova_basa_plot_2.get_xticklabels(), rotation=45, rotation_mode='anchor', ha='right')
plt.title("All specimens")
permanova_basa_plot_2 = permanova_basa_plot_2.get_figure()
plt.tight_layout()
permanova_basa_plot_2.savefig(path_output + 'CART_Microbiome_Fig_2G.pdf')
plt.close()


# Fig_2H:

permanova_table_3_categogeries = ["NHR_Response", "NHS_Survival", "NHR_Progression", "NHR_CRS", "NHR_ICANS"]
permanova_table_3_R2 = permanova_basal_summary_R2_prercentage.loc[permanova_table_3_categogeries,:]
permanova_table_3_R2.index = permanova_table_2_3_category_lables

permanova_table_3_pval_symbols = permanova_basal_summary_pval_symbol.loc[permanova_table_3_categogeries,:]
permanova_table_3_labels =  permanova_table_3_R2.values.astype(str) + np.asarray(permanova_table_3_pval_symbols) 
plt.close()

plt.figure(figsize = (8,6))
sns.set(font_scale=1.5)
permanova_basa_plot_3 = sns.heatmap(permanova_table_3_R2, fmt="", cmap='YlGnBu', annot = permanova_table_3_labels, linewidths=1.5, cbar_kws={'label': 'Variance explained (%)'})
permanova_basa_plot_3.set_xticklabels(permanova_basa_plot_3.get_xticklabels(), rotation=45, rotation_mode='anchor', ha='right')
plt.title("w/o HR abx samples")
permanova_basa_plot_3 = permanova_basa_plot_3.get_figure()
plt.tight_layout()
permanova_basa_plot_3.savefig(path_output + 'CART_Microbiome_Fig_2H.pdf')
plt.close()


### Heatmaps of associations between microbiome and clinical features
* Extended Figures 8A, 8B, 9A and 9B

In [ ]:
def scan_associations(df_X_raw, network_associations_targets_df, abundance_threshold, pval_threshold, foldchange_threshold):
    
    df_X = abundance_filtration(df_X_raw, abundance_threshold)
    
    df_associations_network_pval = pd.DataFrame(index = df_X.columns)
    
    df_associations_network_change = df_associations_network_pval.copy()
    df_associations_network_masked = df_associations_network_pval.copy()

    for current_target in network_associations_targets_df.columns:
        
        current_microbiome_feature_0 = df_X.loc[(network_associations_targets_df[current_target] == 0),:]
        current_microbiome_feature_1 = df_X.loc[(network_associations_targets_df[current_target] == 1),:]
    
        for current_microbiome_feature in df_X.columns:
            
            current_feature_no = current_microbiome_feature_0[current_microbiome_feature]
            current_feature_yes = current_microbiome_feature_1[current_microbiome_feature]   
            
            statistic, pvalue = mannwhitneyu(current_feature_no,current_feature_yes,alternative='two-sided')
            log10_fold_change = np.log10(current_feature_yes.mean() / current_feature_no.mean())
                
            df_associations_network_pval.loc[current_microbiome_feature, current_target] = pvalue
            df_associations_network_change.loc[current_microbiome_feature, current_target] = log10_fold_change
                
            if ( (pvalue > pval_threshold) or (abs(log10_fold_change)) < np.log10(foldchange_threshold)):
                df_associations_network_masked.loc[current_microbiome_feature, current_target] = 0
            else:
                df_associations_network_masked.loc[current_microbiome_feature, current_target] = log10_fold_change
                               
                
    df_associations_network_masked = df_associations_network_masked.fillna(0)
    df_associations_network_masked_abs = abs(df_associations_network_masked)
    df_associations_network_masked_filtered = df_associations_network_masked.loc[df_associations_network_masked_abs.sum(axis = 1) > 0,:]
               
    return(df_associations_network_masked_filtered)


network_associations_targets_df = pd.DataFrame(index = df_CART_table_stool_all.index)

network_associations_targets_df["Age"] = df_colvecs["Age_binarized"]
network_associations_targets_df["Country"] = df_colvecs["Country_yesno"]
network_associations_targets_df["ECOG"] = df_colvecs["ECOG_01"]
network_associations_targets_df["LDH"] = df_colvecs["LDH_Depletion_binarized"]
network_associations_targets_df["Abx."] = df_colvecs["Basal_no_HighRisk_sample"]
network_associations_targets_df["CR"] = df_colvecs["CR180_binarized"]
network_associations_targets_df["Survival"] = df_colvecs["Survival_d180"]
network_associations_targets_df["Progression"] = df_colvecs["Progression_early_binarized"]
network_associations_targets_df["CRS"] = df_colvecs["CRS_01"]

network_associations_targets_df = network_associations_targets_df.replace("no", 0)
network_associations_targets_df = network_associations_targets_df.replace("yes", 1)

network_associations_targets_df = network_associations_targets_df.replace(False, 0)
network_associations_targets_df = network_associations_targets_df.replace(True, 1)

network_associations_targets_df_basal_all = network_associations_targets_df.loc[df_CART_table_stool_basal_all_mean.index,:]
network_associations_targets_df_basal_NoHighRisk = network_associations_targets_df.loc[df_CART_table_stool_basal_NoHighRisk_mean.index,:]

network_associations_targets_df_basal_NoHighRisk = network_associations_targets_df_basal_NoHighRisk.drop(columns='Abx.')

network_associations_targets_df_basal_all.index = df_CART_table_stool_basal_all_mean.subjectID
network_associations_targets_df_basal_NoHighRisk.index = df_CART_table_stool_basal_NoHighRisk_mean.subjectID


# Fig_S8A:
scan_associations_results_taxa_basal_005 = scan_associations(df_CART_table_taxa_basal_all_mean, network_associations_targets_df_basal_all, abundance_threshold = taxa_abundance_threshold, pval_threshold = 0.05, foldchange_threshold = 2)
associations_heatmap_taxa_basal_005 = sns.clustermap(scan_associations_results_taxa_basal_005, cmap = "vlag", vmin = -1, vmax = 1, figsize = (20,80),  col_cluster = False)
associations_heatmap_taxa_basal_005.savefig(path_output + 'CART_Microbiome_Fig_S8A.pdf')
plt.close()

# Fig_S8B:
scan_associations_results_pathways_basal_005 = scan_associations(df_CART_table_pathways_basal_all_mean, network_associations_targets_df_basal_all, abundance_threshold = pathways_abundance_threshold, pval_threshold = 0.05, foldchange_threshold = 2)
associations_heatmap_pathways_basal_005 = sns.clustermap(scan_associations_results_pathways_basal_005, cmap = "vlag", vmin = -1, vmax = 1, figsize = (20,80),  col_cluster = False)
associations_heatmap_pathways_basal_005.savefig(path_output + 'CART_Microbiome_Fig_S8B.pdf')
plt.close()

# Fig_S9A:
scan_associations_results_taxa_basal_005 = scan_associations(df_CART_table_taxa_basal_NoHighRisk_mean, network_associations_targets_df_basal_NoHighRisk, abundance_threshold = taxa_abundance_threshold, pval_threshold = 0.05, foldchange_threshold = 2)
associations_heatmap_taxa_basal_005 = sns.clustermap(scan_associations_results_taxa_basal_005, cmap = "vlag", vmin = -1, vmax = 1, figsize = (20,80),  col_cluster = False)
associations_heatmap_taxa_basal_005.savefig(path_output + 'CART_Microbiome_Fig_S9A.pdf')
plt.close()

# Fig_S9B:
scan_associations_results_pathways_basal_005 = scan_associations(df_CART_table_pathways_basal_NoHighRisk_mean, network_associations_targets_df_basal_NoHighRisk, abundance_threshold = pathways_abundance_threshold, pval_threshold = 0.05, foldchange_threshold = 2)
associations_heatmap_pathways_basal_005 = sns.clustermap(scan_associations_results_pathways_basal_005, cmap = "vlag", vmin = -1, vmax = 1, figsize = (20,80),  col_cluster = False)
associations_heatmap_pathways_basal_005.savefig(path_output + 'CART_Microbiome_Fig_S9B.pdf')
plt.close()



### Processing the ML-derived feature coefficients table as derived for CR prediction

In [ ]:
CR_coefficients_df_filtered = CR_coefficients_df.loc[abs(CR_coefficients_df.Coefficient) > 0,:].copy()
CR_coefficients_df_filtered["Coefficient_absolute"] = abs(CR_coefficients_df_filtered.Coefficient)

features_perc_thr_CR = 25

CR_total_above_perc_thr = round(CR_coefficients_df_filtered.shape[0] * (features_perc_thr_CR  / 100))
CR_coefficients_df_top_perc = CR_coefficients_df_filtered.nlargest(CR_total_above_perc_thr,"Coefficient_absolute")
CR_coefficients_df_top_perc = CR_coefficients_df_top_perc.sort_values(by = "Coefficient")

CR_coefficients_df_top_perc.to_excel(path_output+"CR_coefficients.xlsx", index = False)


### Identifying taxa associations with CD3, CD4, and CD8 T cells blood-levels at apharesis
* Figures 4A and 4B

In [ ]:
apharesis_parameters = ['CD3_mul_Aph', 'CD4_mul_Aph', 'CD8_mul_Aph']

def scan_correlation_associations(df_microbiome, df_Aph, microbiome_features, Aph_parameters, title_name):
    
    if microbiome_features == -1:
        microbiome_features = df_microbiome.columns
    
    df_all_associations = pd.DataFrame(columns=["microbiome_feature", "Aph_parameter", "tau", "pval"])

    for current_Aph_parameter in Aph_parameters: # x
        for current_microbiome_feature in microbiome_features: # y
        
            curr_x_all = df_Aph[current_Aph_parameter]
            curr_y_all = df_microbiome[current_microbiome_feature]
            curr_x = curr_x_all[(np.isnan(curr_x_all.values) == False) & (np.isnan(curr_y_all.values)== False)]
            curr_y = curr_y_all[(np.isnan(curr_x_all.values) == False) & (np.isnan(curr_y_all.values)== False)]
                        
            rho, pval  = stats.kendalltau(curr_x.values , curr_y.values )
       
            current_index = current_Aph_parameter + "---"+current_microbiome_feature
            df_all_associations.loc[current_index, "Aph_parameter"] = current_Aph_parameter
            df_all_associations.loc[current_index, "microbiome_feature"] = current_microbiome_feature
            df_all_associations.loc[current_index, "tau"] = rho
            df_all_associations.loc[current_index, "pval"] = pval
    
    padj_binary, padj =  fdrcorrection(df_all_associations.pval, alpha=0.2, method = 'i')
    
    df_all_associations["Padj"] = padj
    
    df_all_associations_filtered = df_all_associations.loc[df_all_associations.pval < 0.01]
    df_all_associations_sorted = df_all_associations_filtered.sort_values(by = "tau")
    
    df_all_associations_sorted["pvalues_color"] = df_all_associations_sorted.pval
    df_all_associations_sorted.loc[df_all_associations_sorted.pval < 0.01, "pvalues_color"] = "green"
    df_all_associations_sorted.loc[df_all_associations_sorted.Padj < 0.05, "pvalues_color"] = "blue"
    df_all_associations_sorted.loc[df_all_associations_sorted.Padj < 0.01,  "pvalues_color"] = "red"
    
    plt.close()
    plt.barh(y = df_all_associations_sorted.index , width = df_all_associations_sorted.tau, color = df_all_associations_sorted["pvalues_color"])
    plt.xlabel("tau")
    plt.title(title_name)
    plt.savefig(path_output + title_name+".pdf",dpi=300, bbox_inches='tight')
    plt.close()
    
    return(df_all_associations_sorted)

mpl.rcParams.update(mpl.rcParamsDefault)
plt.close()


df_mul_Aph = pd.DataFrame(index=df_CART_table_stool_all.index)
df_mul_Aph["subjectID"] = df_CART_table_stool_all["subjectID"] 

for current_patient in df_mul_Aph.subjectID:
    df_mul_Aph.loc[df_mul_Aph["subjectID"] == current_patient, "CD3_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD3_mul_Aph"].values
    df_mul_Aph.loc[df_mul_Aph["subjectID"] == current_patient, "CD4_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD4_mul_Aph"].values 
    df_mul_Aph.loc[df_mul_Aph["subjectID"] == current_patient, "CD8_mul_Aph"] = df_CART_table_patients_filtered.loc[(df_CART_table_patients_filtered["subjectID"] == current_patient) ,"CD8_mul_Aph"].values

df_mul_Aph_basal_all_mean = df_mul_Aph.loc[df_CART_table_stool_basal_all_mean.index,:]
df_mul_Aph_basal_NoHighRisk_mean = df_mul_Aph.loc[ df_CART_table_stool_basal_NoHighRisk_mean.index,:]

df_mul_Aph_basal_all_mean.index = df_CART_table_stool_basal_all_mean.subjectID
df_mul_Aph_basal_NoHighRisk_mean.index = df_CART_table_stool_basal_NoHighRisk_mean.subjectID


# Fig_4A:
df_apharesis_basal_all_taxa_ML_allTaxa = scan_correlation_associations(df_CART_table_taxa_basal_all_mean, df_mul_Aph_basal_all_mean, -1, apharesis_parameters, "CART_Microbiome_Fig_4A" )


# Fig_4B:
df_apharesis_basal_NoHighRisk_taxa_allTaxa = scan_correlation_associations(df_CART_table_taxa_basal_NoHighRisk_mean, df_mul_Aph_basal_NoHighRisk_mean, -1, apharesis_parameters, "CART_Microbiome_Fig_4B" )



### Comparing the abundance of top ML and differential abundance hits for CR-Abx and Survival stratifications
* Figure 3F
* Extended Figures 7A, 7B, 7C, 7D and 10G (taxa and pathways box-plots)

In [ ]:
def bar_plots_tested(df_microbiome, abundance_threshold, df_colvec, microbiome_feature, colvec_feature,title_name, order, palette, lables, x_title, rotation = 0, showpval = True):
    
    df_microbiome_norm = abundance_filtration(df_microbiome, abundance_threshold)
    
    current_microbiome_feature_0 = df_microbiome_norm.loc[(df_colvec[colvec_feature] == 0),microbiome_feature]
    current_microbiome_feature_1 = df_microbiome_norm.loc[(df_colvec[colvec_feature] == 1),microbiome_feature]
    
    statistic, pvalue = mannwhitneyu(current_microbiome_feature_0,current_microbiome_feature_1,alternative='two-sided')
    
    df_data = pd.concat([np.log10(df_microbiome_norm), df_colvec], axis=1)
    
    df_data_untransformed = pd.concat([df_microbiome_norm, df_colvec], axis=1)
    
    y_parameter = microbiome_feature
    x_parameter = colvec_feature
    
    np.random.seed(1)
    plt.figure(figsize = (2,5))
    sns.set(font_scale=0.8)
    sns.set_style("ticks")
    curr_plt = sns.boxplot(x= x_parameter, y = y_parameter, data = df_data, color = "whitesmoke",showfliers = False, order = order)
    sns.stripplot(x = x_parameter, y = y_parameter, data = df_data, order = order, palette = palette)
    plt.setp(curr_plt.get_xticklabels(), rotation=0)
    
    curr_plt.set_ylim([np.log10(abundance_threshold)-0.2, 0])   
    
    curr_plt.set_xticklabels(lables, rotation = rotation)
    
    if abundance_threshold == 1e-3:
        curr_plt.set_yticks([-3, -2, -1, 0])
        curr_plt.set_yticklabels(['1e-3', '1e-2', '1e-1', '1'])
        
    if abundance_threshold == 1e-4:
        curr_plt.set_yticks([-4, -3, -2, -1, 0])
        curr_plt.set_yticklabels(['1e-4', '1e-3', '1e-2', '1e-1', '1'])
   
    if showpval:
        plt.title(microbiome_feature + "\n" + "p  = " +str(np.round(pvalue,5)),fontsize = 12)
    else:
        plt.title(microbiome_feature,fontsize = 12)
        
    plt.xlabel(x_title,fontsize = 12)
    plt.ylabel("Relative abundance",fontsize = 12)
    plt.savefig(path_output + title_name + "_" + microbiome_feature +".pdf",dpi=300, bbox_inches='tight')
    plt.close()
    
    if showpval == False:
        
        df_Pvals = pd.DataFrame()
        
        file_object = open(path_output + title_name+"_"+ microbiome_feature + "_Pvals.txt", 'a')
    
        file_object.write (title_name+"_"+ microbiome_feature + "_"+ colvec_feature)
        file_object.write("\n ")
    
        for paramater_a in order:
            current_a_values = df_data_untransformed.loc[df_data_untransformed[x_parameter] == paramater_a, y_parameter]
        
            for paramater_b in order[order.index(paramater_a)+1:]:
                current_b_values = df_data_untransformed.loc[df_data_untransformed[x_parameter] == paramater_b, y_parameter]
            
                statistic, pvalue = mannwhitneyu(current_a_values, current_b_values, alternative='two-sided')
                
                current_ID = paramater_a + "------" + paramater_b
                df_Pvals.loc[current_ID, "Pval"] = pvalue  
                
        df_Pvals.index.name='Compared parameters'    
                
        padj_binary, padj =  fdrcorrection(df_Pvals.Pval, alpha=0.2, method = 'i')
        df_Pvals.loc[:, "Padjusted"] = padj
        file_object.write("\n ")
        
        print(df_Pvals, file=file_object)
  
        file_object.close()
    
    return


# Fig_3F:
Taxa_to_assess_Fig_3F = ["Bacteroides_eggerthii", "Ruminococcus_lactaris", "Eubacterium_sp_CAG_180", "Akkermansia_muciniphila", "Erysipelatoclostridium_ramosum"]
for current_taxon in Taxa_to_assess_Fig_3F:
    bar_plots_tested(df_CART_table_taxa_basal_all_mean, taxa_abundance_threshold, df_colvecs_basal_all_mean, current_taxon, "CR180_noHRAbx", "CART_Microbiome_Fig_3F", 
                     order = ['noHR + CR',  "noHR + noCR","HR + CR", "HR + noCR"], 
                     palette = {'noHR + CR': "blue", "noHR + noCR": "orange", "HR + CR":"red", "HR + noCR":"green" }, 
                     lables = ['noHR + CR',  "noHR + noCR","HR + CR", "HR + noCR"], x_title= "", rotation = 90, showpval = False)


# Fig_S10G:
Taxa_to_assess_Fig_S10G = ["Bacteroides_stercoris", "Bacteroides_fragilis", "Roseburia_faecis", "Eubacterium_sp_CAG_38" ]
for current_taxon in Taxa_to_assess_Fig_S10G:
    bar_plots_tested(df_CART_table_taxa_basal_all_mean, taxa_abundance_threshold, df_colvecs_basal_all_mean, current_taxon, "CR180_noHRAbx", "CART_Microbiome_Fig_S10G", 
                     order = ['noHR + CR',  "noHR + noCR","HR + CR", "HR + noCR"], 
                     palette = {'noHR + CR': "blue", "noHR + noCR": "orange", "HR + CR":"red", "HR + noCR":"green" }, 
                     lables = ['noHR + CR',  "noHR + noCR","HR + CR", "HR + noCR"], x_title= "", rotation = 90, showpval = False)
    

# Fig_S7A (bacteria bar-plots):
Taxa_to_assess_Fig_S7A = ["Bifidobacterium_longum"]
for current_taxon in Taxa_to_assess_Fig_S7A:
    bar_plots_tested(df_CART_table_taxa_basal_all_mean, taxa_abundance_threshold, df_colvecs_basal_all_mean, current_taxon, "Survival_d180", "CART_Microbiome_Fig_S7A", 
                     order = [1, 0], palette = {1: "green", 0: "red"}, lables = [">180", "<180"], x_title= "Survival (days)")

    
# Fig_S7B (bacteria bar-plots):
Taxa_to_assess_Fig_S7B = ["Bifidobacterium_longum"]
for current_taxon in Taxa_to_assess_Fig_S7B:
    bar_plots_tested(df_CART_table_taxa_basal_NoHighRisk_mean, taxa_abundance_threshold, df_colvecs_basal_NoHighRisk_mean, current_taxon, "Survival_d180", "CART_Microbiome_Fig_S7B", 
                     order = [1, 0], palette = {1: "green", 0: "red"}, lables = [">180", "<180"], x_title= "Survival (days)")

    
# Fig_S7C (pathways bar-plots):
Taxa_to_assess_Fig_S7C = ["PWY-5265: peptidoglycan biosynthesis II (staphylococci)"]
for current_taxon in Taxa_to_assess_Fig_S7C:
    bar_plots_tested(df_CART_table_pathways_basal_NoHighRisk_mean, pathways_abundance_threshold, df_colvecs_basal_NoHighRisk_mean, current_taxon, "Survival_d180", "CART_Microbiome_Fig_S7C", 
                     order = [1, 0], palette = {1: "green", 0: "red"}, lables = [">180", "<180"], x_title= "Survival (days)")

    
# Fig_S7D (pathways bar-plots):
Taxa_to_assess_Fig_S7D = ["PWY-5265: peptidoglycan biosynthesis II (staphylococci)"]
for current_taxon in Taxa_to_assess_Fig_S7D:
    bar_plots_tested(df_CART_table_pathways_basal_NoHighRisk_mean, pathways_abundance_threshold, df_colvecs_basal_NoHighRisk_mean, current_taxon, "Progression_early_binarized", "CART_Microbiome_Fig_S7D", 
                     order = [0, 1], palette = {1: "red", 0: "green"}, lables = ["NO | >180 days", "<180 days"], x_title= "")

    

### Comparing T-cells, CRP and IL-6 levels for high/low-abudance taxa stratification
* Figures 4C, 4D and 4E

In [ ]:
def compare_abundance_startified(df_microbiome, df_colvec, microbiome_feature, colvec_feature, title_name):
    
    curr_x_all = df_microbiome[microbiome_feature] 
    curr_y_all = df_colvec[colvec_feature]
    
    curr_x = curr_x_all[~np.isnan(curr_x_all) & ~np.isnan(curr_y_all)]
    curr_y = curr_y_all[~np.isnan(curr_x_all) & ~np.isnan(curr_y_all)]
    
    startifing_threshold = np.quantile(curr_x, 0.5)
    
    # Stratifying based on taxa abundance and performing mannwhitneyu test:

    df_data = pd.DataFrame()
    df_data["x_parameter"] = (curr_x <= startifing_threshold)
    df_data["y_parameter"] = curr_y
    
    colvec_feature_below = curr_y[curr_x <= startifing_threshold]
    colvec_feature_above = curr_y[curr_x > startifing_threshold]
    
    statistic, pvalue = mannwhitneyu(colvec_feature_below,colvec_feature_above,alternative='two-sided')
    
    np.random.seed(1)
    plt.figure(figsize = (2,5))
    sns.set(font_scale=0.8)
    sns.set_style("ticks")
    
    curr_plt = sns.boxplot(x= "x_parameter", y = "y_parameter", data = df_data, color = "whitesmoke",showfliers = False, order = [True, False])
    sns.stripplot(x = "x_parameter", y = "y_parameter", data = df_data, order = [True, False], palette = {1: "green", 0: "red"})
    plt.setp(curr_plt.get_xticklabels(), rotation=0)
  
    plt.title(title_name + "\n" + microbiome_feature + "\n" + "mannwhitneyu Pval  = " + str(np.round(pvalue,5)),fontsize = 12)
    curr_plt.set_xticklabels(["Low", "High"])
    plt.xlabel(microbiome_feature)
    plt.ylabel(colvec_feature, fontsize = 12)
  
    plt.savefig(path_output + title_name + "_" + microbiome_feature +"_"+colvec_feature +"_stratified.pdf",dpi=300, bbox_inches='tight')
    plt.close()
    
    return


# Fig_4C:
compare_abundance_startified(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean, "Lachnospira_pectinoschiza", 'CD3_mul_Aph',  "CART_Microbiome_Fig_4C")
compare_abundance_startified(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean, "Akkermansia_muciniphila", 'CD4_mul_Aph',  "CART_Microbiome_Fig_4C")
compare_abundance_startified(df_CART_table_taxa_basal_NoHighRisk_mean, df_colvecs_basal_NoHighRisk_mean, "Firmicutes_bacterium_CAG_424", 'CD4_mul_Aph',  "CART_Microbiome_Fig_4C")


# Fig_4D:
compare_abundance_startified(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean, "Lachnospira_pectinoschiza", 'CRP_depletion',  "CART_Microbiome_Fig_4D")
compare_abundance_startified(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean, "Bacteroides_eggerthii", 'CRP_depletion',  "CART_Microbiome_Fig_4D")
compare_abundance_startified(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean, "Ruminococcus_lactaris", 'CRP_depletion',  "CART_Microbiome_Fig_4D")


# Fig_4E:
compare_abundance_startified(df_CART_table_taxa_basal_all_mean, df_colvecs_basal_all_mean, "Lachnospira_pectinoschiza", 'IL6_depletion',  "CART_Microbiome_Fig_4E")



### Alpha-diversity : Scatter plots of Shannon index versus relative sampling day

* Figures 2A and 2B-(left)

In [ ]:
def moving_median(x, y, window_size):
    
    x = np.array(x)
    y = np.array(y)
    
    x_start = min(x)
    x_end = max(x)
    
    frames = range(int(x_start + np.ceil(window_size/2)), int(x_end - np.floor(window_size/2)))
    
    moving_average_line = pd.DataFrame(index=frames, columns=["x", "y"])
    
    for window_middle in frames:
        
        window_start = window_middle - window_size/2
        window_end = window_middle + window_size/2
        
        current_y = y[(x > window_start) & (x < window_end )]
        
        if len(current_y) > 4:
            
            moving_average_line.loc[window_start, "x"] = window_middle
            moving_average_line.loc[window_start, "y"] = np.median(current_y)
       
    return(moving_average_line)


curr_x_all = df_CART_table_stool_all.sampleDay_sample
curr_y_all = df_Shannon.Shannon

curr_x_lim = [-20, 70]
curr_y_lim = [curr_y_all.min()-0.1, curr_y_all.max()+0.1]

curr_x = curr_x_all[(curr_x_all > curr_x_lim[0] ) & (curr_x_all < curr_x_lim[1] )]
curr_y = curr_y_all[(curr_x_all > curr_x_lim[0] ) & (curr_x_all < curr_x_lim[1] )]

curr_x_noHR = curr_x[df_CART_table_stool_all.AbxRisk_On_Sampling != "HR"]
curr_y_noHR = curr_y[df_CART_table_stool_all.AbxRisk_On_Sampling != "HR"]

curr_x_HR = curr_x[df_CART_table_stool_all.AbxRisk_On_Sampling == "HR"]
curr_y_HR = curr_y[df_CART_table_stool_all.AbxRisk_On_Sampling == "HR"]

    

# Fig_2A:

plt.figure(figsize = (5,5))

plt.scatter(curr_x, curr_y, c= "gray", s = 3)

moving_median_line = moving_median(curr_x, curr_y, 7)
x1 = moving_median_line.x[~np.isnan(list(moving_median_line.x.values)) &~np.isnan(list(moving_median_line.y.values))]
y1 = moving_median_line.y[~np.isnan(list(moving_median_line.x.values)) &~np.isnan(list(moving_median_line.y.values))]
line_model = make_interp_spline(x1,y1, k=1)
xs=np.linspace(min(x1),max(x1),round((max(x1) - min(x1)) / 4))
ys=line_model(xs)
plt.plot(xs, ys, c= "black")

plt.xlim(curr_x_lim)
plt.ylim(curr_y_lim)
plt.title("CART_Microbiome_Fig_2A")
plt.xlabel("Days relative to CAR-T infusion")
plt.ylabel("Shannon index")
plt.savefig(path_output + 'CART_Microbiome_Fig_2A.pdf',dpi=300, bbox_inches='tight')
plt.close()
        

# Fig_2B_left: 

plt.figure(figsize = (5,5))

plt.scatter(curr_x_noHR, curr_y_noHR, c= "green", s = 3, label='None | LR')
plt.scatter(curr_x_HR, curr_y_HR, c= "red", s = 3, label='HR')

moving_median_line_noHR = moving_median(curr_x_noHR, curr_y_noHR, 7)
x1 = moving_median_line_noHR.x[~np.isnan(list(moving_median_line_noHR.x.values)) &~np.isnan(list(moving_median_line_noHR.y.values))]
y1 = moving_median_line_noHR.y[~np.isnan(list(moving_median_line_noHR.x.values)) &~np.isnan(list(moving_median_line_noHR.y.values))]
line_model = make_interp_spline(x1,y1, k=1)
xs=np.linspace(min(x1),max(x1),round((max(x1) - min(x1)) / 4))
ys=line_model(xs)
plt.plot(xs, ys, c= "green")

moving_median_line_HR = moving_median(curr_x_HR, curr_y_HR, 7)
x1 = moving_median_line_HR.x[~np.isnan(list(moving_median_line_HR.x.values)) &~np.isnan(list(moving_median_line_HR.y.values))]
y1 = moving_median_line_HR.y[~np.isnan(list(moving_median_line_HR.x.values)) &~np.isnan(list(moving_median_line_HR.y.values))]
line_model = make_interp_spline(x1,y1, k=1)
xs=np.linspace(min(x1),max(x1),round((max(x1) - min(x1)) / 4))
ys=line_model(xs)
plt.plot(xs, ys, c= "red")

plt.xlim(curr_x_lim)
plt.ylim(curr_y_lim)
plt.title("CART_Microbiome_Fig_2B")
plt.xlabel("Days relative to CAR-T infusion")
plt.ylabel("Shannon index")
plt.legend()
plt.savefig(path_output + 'CART_Microbiome_Fig_2B_left.pdf',dpi=300, bbox_inches='tight')
plt.close()



### Alpha-diversity: Box-plots comparing Shannon index between therapy outcomes
* Figure 2C
* Extended Figures 5B (left and right), 5C and 5D

In [ ]:
def bar_plots_Shannon(df_colvec, colvec_feature, title_name, order, palette, lables, x_title, rotation = 0, showpval = True ):
    
    current_microbiome_feature_0 = df_colvec.loc[(df_colvec[colvec_feature] == 0), "Shannon"]
    current_microbiome_feature_1 = df_colvec.loc[(df_colvec[colvec_feature] == 1), "Shannon"]
    
    statistic, pvalue = mannwhitneyu(current_microbiome_feature_0,current_microbiome_feature_1,alternative='two-sided')
    
    y_parameter = "Shannon"
    x_parameter = colvec_feature
    
    np.random.seed(1)
    plt.figure(figsize = (2,5))
    sns.set(font_scale=0.8)
    sns.set_style("ticks")
    curr_plt = sns.boxplot(x= x_parameter, y = y_parameter, data = df_colvec, color = "whitesmoke",showfliers = False, order = order)
    sns.stripplot(x = x_parameter, y = y_parameter, data = df_colvec, order = order, palette = palette)
    plt.setp(curr_plt.get_xticklabels(), rotation=0)
 
    curr_plt.set_xticklabels(lables, rotation = rotation)
   
    if showpval:
        plt.title("Shannon" + "\n" + "p  = " +str(np.round(pvalue,5)),fontsize = 12)
    else:
        plt.title("Shannon",fontsize = 12)
        
    plt.xlabel(x_title,fontsize = 12)
    plt.ylabel("Shannon index",fontsize = 12)
    plt.savefig(path_output + title_name + "_" + "Shannon_"+  colvec_feature  +".pdf",dpi=300, bbox_inches='tight')
    plt.close()
    
    if showpval == False:
        
        df_Pvals = pd.DataFrame()
        
        file_object = open(path_output + title_name+"_"+ "Shannon_" +  colvec_feature + "_Pvals.txt", 'a')
    
        file_object.write (title_name+"_"+ "Shannon" + "_"+ colvec_feature)
        file_object.write("\n ")
    
        for paramater_a in order:
            current_a_values = df_colvec.loc[df_colvec[x_parameter] == paramater_a, y_parameter]
        
            for paramater_b in order[order.index(paramater_a)+1:]:
                current_b_values = df_colvec.loc[df_colvec[x_parameter] == paramater_b, y_parameter]
            
                statistic, pvalue = mannwhitneyu(current_a_values, current_b_values, alternative='two-sided')
                
                current_ID = str(paramater_a) + "------" + str(paramater_b)
                df_Pvals.loc[current_ID, "Pval"] = pvalue  
                        
        df_Pvals.index.name='Compared parameters'    
                
        padj_binary, padj =  fdrcorrection(df_Pvals.Pval, alpha=0.2, method = 'i')
        df_Pvals.loc[:, "Padjusted"] = padj
        file_object.write("\n ")
        
        print(df_Pvals, file=file_object)
        file_object.write("\n ")
        file_object.close()

    return

# Fig_2C (left and right):
bar_plots_Shannon(df_colvecs_basal_all_mean, "CR180_binarized", "CART_Microbiome_Fig_2C_left",  order = [1, 0], palette = {1: "blue", 0: "orange"}, lables = ["CR", "nonCR"], x_title= "")
bar_plots_Shannon(df_colvecs_basal_all_mean, "Progression_early_binarized", "CART_Microbiome_Fig_2C_right",  order = [0, 1], palette = {1: "orange", 0: "blue"}, lables = ["No | > 180 days", "< 180 days"], x_title= "")
      
    
# Fig_S5B (left and right):
bar_plots_Shannon(df_colvecs_basal_NoHighRisk_mean, "CR180_binarized", "CART_Microbiome_Fig_S5B_left",  order = [1, 0], palette = {1: "blue", 0: "orange"}, lables = ["CR", "nonCR"], x_title= "Response")
bar_plots_Shannon(df_colvecs_basal_NoHighRisk_mean, "Progression_early_binarized", "CART_Microbiome_Fig_S5B_right",  order = [0, 1], palette = {1: "orange", 0: "blue"}, lables = ["No | > 180 days", "< 180 days"], x_title= "Early progression")

# Fig_S5C
bar_plots_Shannon(df_colvecs_basal_NoHighRisk_mean, "CRS_012_34", "CART_Microbiome_Fig_S5C",  order = [0, 1, 2, 34], palette = {0: "orange", 1: "blue", 2: "red", 34:"gray"}, lables = ["0", "1", "2", "34"], x_title= "CRS", showpval = False)

# Fig_S5D:
bar_plots_Shannon(df_colvecs_basal_NoHighRisk_mean, "ICANS_012_34", "CART_Microbiome_Fig_S5D",  order = [0, 1, 2, 34], palette = {0: "orange", 1: "blue", 2: "red", 34:"gray"}, lables = ["0", "1", "2", "34"], x_title= "ICANS", showpval = False)



### Alpha-diversity: Per-patient comparison of Shannon indices before and after high-risk antibiotics
* Figure 2B-right

In [ ]:
df_stool_samples_risk_pairing = df_CART_table_stool_all.copy()

for current_patient in list(set(df_CART_table_stool_all.subjectID)):
    
    current_patint_samples_df = df_CART_table_stool_all.loc[df_CART_table_stool_all.subjectID == current_patient]
    current_patint_samples_df = current_patint_samples_df.sort_values(by = "sampleDay_sample", ascending=True)
    
    patient_sample_types = current_patint_samples_df.AbxRisk_On_Sampling.values
    
    patient_sample_types_list = list(patient_sample_types)
    
    if ((sum(patient_sample_types == "HR") > 0) & (sum(patient_sample_types != "HR") > 0)):
        first_HR_sample = patient_sample_types_list.index("HR")
        
        if (first_HR_sample > 0):

            patinet_preHR_samples = list(current_patint_samples_df.stool_sample_id)[0:first_HR_sample]
            patinet_HR_samples = list(current_patint_samples_df.stool_sample_id[current_patint_samples_df.AbxRisk_On_Sampling == "HR"])
            
            df_stool_samples_risk_pairing.loc[patinet_preHR_samples, "LongitudinalRiskType"] = "noHR"
            df_stool_samples_risk_pairing.loc[patinet_HR_samples, "LongitudinalRiskType"] = "HR"
        
paired_noHR_samples = df_stool_samples_risk_pairing.LongitudinalRiskType == "noHR"
paired_HR_samples = df_stool_samples_risk_pairing.LongitudinalRiskType == "HR"

df_taxa_risk_paired_mean_noHR,  df_stool_risk_paired_mean_noHR = derive_mean_microbiome(df_CART_table_taxa_all_norm.loc[paired_noHR_samples] , df_CART_table_stool_all.loc[paired_noHR_samples])
df_taxa_risk_paired_mean_HR, df_stool_risk_paired_mean_HR =     derive_mean_microbiome(df_CART_table_taxa_all_norm.loc[paired_HR_samples] , df_CART_table_stool_all.loc[paired_HR_samples])

df_Shannon_risk_paired = pd.DataFrame(index=df_taxa_risk_paired_mean_noHR.index, columns=["noHR", "HR"])

for current_subject in df_taxa_risk_paired_mean_noHR.index:
    df_Shannon_risk_paired.loc[current_subject, "noHR"] = shannon(df_taxa_risk_paired_mean_noHR.loc[current_subject,:])
    df_Shannon_risk_paired.loc[current_subject, "HR"] = shannon(df_taxa_risk_paired_mean_HR.loc[current_subject,:])
    
statistic_sh, pvalue_sh = wilcoxon(df_Shannon_risk_paired.noHR, df_Shannon_risk_paired.HR)

# Fig. 2B-right:
np.random.seed(1)
plt.figure(figsize = (2,5))
sns.set(font_scale=0.8)
sns.set_style("ticks")
curr_plt = sns.boxplot(data = df_Shannon_risk_paired, color = "whitesmoke",showfliers = False)
sns.stripplot(data = df_Shannon_risk_paired, palette = {"noHR": "black", "HR": "red"})
plt.setp(curr_plt.get_xticklabels(), rotation=0)
plt.title("Pval = " +  str(np.round(pvalue_sh,5)) + " (n = " + str(df_Shannon_risk_paired.shape[0])+ ")")
plt.ylabel("Shannon index (mean per patient and risk type)")
plt.savefig(path_output + "CART_Microbiome_Fig_2B_right.pdf",dpi=300, bbox_inches='tight')
plt.close()



### Plots of the number of samples per relative sampling day
* Extended Figure 5A

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
earlies_day = df_colvecs["day_colvec"].min()
latest_day = df_colvecs["day_colvec"].max()
all_centers = set(df_colvecs.Center)

days_vec = list(range(earlies_day,latest_day+1))

df_samples_per_day = pd.DataFrame(columns=all_centers, index = days_vec)
    
for current_center in all_centers:
    for current_day in days_vec:
        df_samples_per_day.loc[current_day, current_center] = sum((df_colvecs["Center"] == current_center) & (df_colvecs["day_colvec"] == current_day))
            
fig, (ax1,ax2) = plt.subplots(nrows=2, sharex=True, sharey=True,figsize=(8, 5), subplot_kw=dict(frameon=True))

sbar_width = 0.8

ax1.grid(color='gray', linestyle='--', linewidth=0.3)
ax1.bar(x = days_vec, height = df_samples_per_day.HD, color ="red", width=sbar_width, label="Heidelberg")
ax1.bar(x = days_vec, height = df_samples_per_day.LMU, color ="blue", width=sbar_width, label="Munich")
ax1.bar(x = days_vec, height = df_samples_per_day.UKR,color ="orange", width=sbar_width,  label="Regensburg")
ax1.legend()
ax1.set_ylim([0,20])
 
    
ax2.grid(color='gray', linestyle='--', linewidth=0.3)
ax2.bar(x = days_vec, height = df_samples_per_day.MDACC, color ="green", width=sbar_width, label="MDACC")
ax2.bar(x = days_vec, height = df_samples_per_day.MOFFITT, color ="purple", width=sbar_width, label="Moffitt")
ax2.legend()
ax2.set_ylim([0,20])
  
fig.text(-0.01, 0.5, 'Number of stool samples collected', va='center', rotation='vertical', fontsize = 12)
    
fig.text(0.08, 0.91, 'Germany', va='center', rotation='horizontal', fontsize = 14, bbox=dict(facecolor='lightgray', edgecolor='gray', pad= 5))
fig.text(0.08, 0.46, 'US', va='center', rotation='horizontal', fontsize = 14, bbox=dict(facecolor='lightgray', edgecolor='gray', pad= 5))
    
    
plt.xlabel("Days relative to CAR-T cell infusion", fontsize = 12)
plt.tight_layout()
plt.savefig(path_output + "CART_Microbiome_Fig_S5A.pdf",dpi=300, bbox_inches='tight')
plt.close()
